In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import date
import questionary

In [ ]:
stocks = ['AAPL', 'AMZN', 'FB', 'TSLA', '^GSPC']

start_date = '2010-01-01'

end_date = '2017-01-01'

data = {}

for stock in stocks:
    ticker = yf.download(stock, start_date, end_date)
    # Adjusted Closing Price takes into account factors such as dividends, stock splits, etc.
    # Adjusted Closing Price is a more accurate reflection of the stock's value
    data[stock] = ticker['Adj Close']

pd.DataFrame(data)

In [ ]:
stocks_data = pd.DataFrame(data)

stocks_data = stocks_data.fillna(stocks_data.rolling(6, min_periods=1).mean())
stocks_data = stocks_data.dropna()

stocks_data = stocks_data.resample('M').last()
stocks_data

In [ ]:
sp500 = stocks_data.iloc[:,-1]
sp500 = pd.DataFrame(sp500)

In [ ]:
sp500.head()

In [ ]:
sp500['market_adjclose'] = np.log(sp500 / sp500.shift(1))
sp500 = sp500[1:]
sp500.head()

In [ ]:
stocks = stocks_data.iloc[:,:-1]
stocks = np.log(stocks / stocks.shift(1))
stocks = stocks.dropna()
stocks = pd.DataFrame(stocks)

stocks.head()

In [ ]:
weights = [0.25, 0.3, 0.1, 0.4]
weighted_stocks = stocks.mul(weights)
weighted_stocks.head()


In [ ]:
# we have to have a sector return value to compare against the market return
# so we sum up all the stocks in the sector per day to figure out the daily_return compared to the market
weighted_stocks['sector_return'] = weighted_stocks.sum(axis=1)
weighted_stocks.head()

In [ ]:
weights = [0.25, 0.3, 0.1, 0.4]
print(weights)

In [ ]:
data = pd.DataFrame({'stock_weighted_returns':weighted_stocks['sector_return'], 'market_returns':sp500['market_adjclose']})
data.head()

In [ ]:
    def initialize(self):
            stocks_data = self.download_data()
            # we use monthly returns ('M') instead of daily returns
            stocks_data = stocks_data.resample('M').last()

            # create a pandas dataframe to store stock information for analysis
            self.data = pd.DataFrame({'stock_adjclose':stocks_data[self.stocks[0]], 'market_adjclose':stocks_data[self.stocks[-1]]})

            # add 2 columns for the s_returns and m_returns
            # logarithmic monthly returns
            self.data[['stock_returns', 'market_returns']] = np.log(self.data[['stock_adjclose', 'market_adjclose']] / self.data[['stock_adjclose', 'market_adjclose']].shift(1))

            # remove NaN values
            self.data =  self.data[1:]

            print(self.data)